# Load & delete an on-prem instance

Here, we construct a case in which the storage location of the previous instance was moved:

In [ ]:
!lamin close
!yes | lamin delete mydata
!lamin init --storage mydata
!rm -r ./mydata_new_loc
!mv mydata ./mydata_new_loc
!lamin close

In [ ]:
import lamindb_setup as ln_setup

## Load your own instance by name

If the user is the instance owner, load the instance by name:
```
ln_setup.connect("mydata")
```

You can also load with a new default storage location:

In [ ]:
ln_setup.connect("mydata", storage="./mydata_new_loc")

In [ ]:
from pathlib import Path

assert ln_setup.settings.instance.storage.is_cloud == False
assert ln_setup.settings.instance.name == "mydata"
assert (
    ln_setup.settings.instance.storage.root.as_posix()
    == Path("./mydata_new_loc").resolve().as_posix()
)
assert (
    ln_setup.settings.instance.db
    == f"sqlite:///{Path('./mydata_new_loc').resolve().as_posix()}/{ln_setup.settings.instance.id.hex}.lndb"
)

You cannot load another instance in the same Python session:

In [ ]:
import pytest

with pytest.raises(RuntimeError):
    ln_setup.connect("testuser2/mydata")

assert ln_setup.settings.instance.slug == "testuser1/mydata"

## Delete an instance

In [ ]:
assert ln_setup.settings._instance_settings_path.exists()
ln_setup.delete("mydata", force=True)
assert not ln_setup.settings._instance_settings_path.exists()